# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data= 'Cities_output.csv'
city_df= pd.read_csv(city_data)
city_df.head(10)

,City,Country,Date,Latitude,Longitude,Maximum Temperature,Humidity,Cloudiness,Wind Speed
0,Faanui,PF,1587445574,-16.48,-151.75,81.73,79,99,17.02
1,Lębork,PL,1587445574,54.54,17.75,42.01,70,1,5.84
2,Hithadhoo,MV,1587445485,-0.60,73.08,85.03,69,100,5.73
3,Ushuaia,AR,1587445551,-54.80,-68.30,42.80,75,40,38.03
4,Atuona,PF,1587445575,-9.80,-139.03,81.79,74,24,13.33
5,Port-Cartier,CA,1587445575,50.03,-66.87,25.00,41,5,9.17
6,La Providencia,MX,1587445575,28.72,-111.58,73.99,40,97,1.99
7,Port Elizabeth,ZA,1587445495,-33.92,25.57,59.00,87,59,6.93
8,Ulladulla,AU,1587445576,-35.35,150.47,77.00,38,0,8.05
9,Upernavik,GL,1587445576,72.79,-56.15,25.66,87,99,9.80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
coordinates= city_df[["Latitude","Longitude"]]
humidity= city_df['Humidity'].astype(float)

In [35]:
vacay_heatmap=gmaps.figure()
layer=gmaps.heatmap_layer(coordinates, weights= humidity, dissipating= False)
vacay_heatmap.add_layer(layer)
vacay_heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_df=pd.DataFrame(city_df)
new_df.head()
temp= (new_df['Maximum Temperature']>70) & (new_df['Maximum Temperature']<80)
wind= (new_df['Wind Speed']<10)
cloud= (new_df['Cloudiness']== 0)
new_df= new_df[temp & wind & cloud]
new_df=new_df.dropna()
new_df

,City,Country,Date,Latitude,Longitude,Maximum Temperature,Humidity,Cloudiness,Wind Speed
8,Ulladulla,AU,1587445576,-35.35,150.47,77.00,38,0,8.05
61,Batemans Bay,AU,1587445587,-35.72,150.18,71.01,62,0,5.01
64,Umluj,SA,1587445588,25.02,37.27,79.61,41,0,0.40
118,Ürümqi,CN,1587445475,43.80,87.60,71.60,24,0,2.24
129,Ushtobe,KZ,1587445604,45.25,77.98,75.76,20,0,3.00
149,Araouane,ML,1587445609,18.90,-3.53,72.01,19,0,8.37
185,Tessalit,ML,1587445618,20.20,1.01,71.92,21,0,5.70
369,Sabha,LY,1587445663,27.04,14.43,73.00,24,0,7.20
521,Ubari,LY,1587445701,26.59,12.78,70.30,24,0,5.21
545,Koson,UZ,1587445706,39.04,65.58,71.60,56,0,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df= pd.DataFrame(new_df, columns= ['City','Country','Latitude','Longitude'])
# hotel_df=hotel_df.assign(Hotel_Name= [])
hotel_df['Hotel Name']=''
hotel_df


,City,Country,Latitude,Longitude,Hotel Name
8,Ulladulla,AU,-35.35,150.47,
61,Batemans Bay,AU,-35.72,150.18,
64,Umluj,SA,25.02,37.27,
118,Ürümqi,CN,43.80,87.60,
129,Ushtobe,KZ,45.25,77.98,
149,Araouane,ML,18.90,-3.53,
185,Tessalit,ML,20.20,1.01,
369,Sabha,LY,27.04,14.43,
521,Ubari,LY,26.59,12.78,
545,Koson,UZ,39.04,65.58,


In [30]:
# url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel= []

parameters = {
    "radius": 5000,
    "keyword": "hotel",
    "types":"hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat= row["Latitude"]
    long=row["Longitude"]
    parameters['location']= f'{lat},{long}'
    url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response= requests.get(url,params=parameters)
    response2= response.json()
    try:
        hotel.append(response2['results'][0]['name'])
    except IndexError:
        hotel.append(np.nan)

In [32]:
hotel_df["Hotel Name"]= hotel
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
8,Ulladulla,AU,-35.35,150.47,Allure on Ocean Motel (formerly Mollymook Seas...
61,Batemans Bay,AU,-35.72,150.18,Corrigans Cove Resort
64,Umluj,SA,25.02,37.27,عروس الشاطئ للوحدات السكنية المفروشة
118,Ürümqi,CN,43.80,87.60,JinJiang International Hotel Urumqi
129,Ushtobe,KZ,45.25,77.98,NaN
149,Araouane,ML,18.90,-3.53,NaN
185,Tessalit,ML,20.20,1.01,NaN
369,Sabha,LY,27.04,14.43,فندق المواجيد
521,Ubari,LY,26.59,12.78,NaN
545,Koson,UZ,39.04,65.58,NaN


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [37]:
# Add marker layer ontop of heat map
vacay_heatmap.add_layer(gmaps.marker_layer(locations))

# Display figure
vacay_heatmap

Figure(layout=FigureLayout(height='420px'))